In [1]:
import importlib
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

def check_and_install(libraries):
    for lib in libraries:
        try:
            importlib.import_module(lib)
            print(f"{lib} is already installed.")
        except ImportError:
            print(f"{lib} is not installed. Installing now...")
            install_package(lib)
            print(f"{lib} has been successfully installed.")

# List of libraries to check and install
libraries_to_check = ['stable_baselines3', 'torch', 'matplotlib', 'gdown']

check_and_install(libraries_to_check)

2024-08-01 01:21:00.123005: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 01:21:00.123081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 01:21:00.124494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 01:21:00.133757: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 01:21:01.360133: W tensorflow/compiler/tf2

stable_baselines3 is already installed.
torch is already installed.
matplotlib is already installed.
gdown is already installed.


In [2]:
import torch
from typing import Optional, Dict
from dataclasses import dataclass
import random
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from IPython.display import clear_output
import matplotlib.colors as mcolors

import gymnasium as gym
import torch
import numpy as np
from collections import defaultdict
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import Figure
tensorboard_log = './ppotb'
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
!pwd

/notebooks


In [4]:

def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Breeding Simulator

In [5]:

## BREEDING SIMULATOR
    

@dataclass
class SimulationConfig:
    #training
    total_timesteps: int = 2000000
    seed: int = 1
    #breeding sim parameters
    n_markers: int = 1000
    starting_parents: int = 2
    pop_size: int = 1000
    h2: float = 0.3
    max_generations:int = 6
    sparse_reward: bool = True
    budget_per_cycle: int = 200
    starting_budget: int = max_generations * budget_per_cycle
    min_offspring: int = 20
    selection_subpops: int = 2
    fixed_budget: bool = False

config = SimulationConfig()

class Genome:
    def __init__(self, n_markers: int):
        self.ploidy: int = 2
        self.n_markers: int = n_markers

    def __repr__(self) -> str:
        return f"Genome(ploidy={self.ploidy}, n_markers={self.n_markers})"

class Population:
    def __init__(self, pop_size: int, haplotypes:torch.tensor, genome: Genome, device: torch.device):
        self.pop_size: int = pop_size
        self.genome: Genome = genome
        self.haplotypes: torch.Tensor = haplotypes
        self.device: torch.device = device
        
    def to(self, device: torch.device) -> 'Population':
        self.device = device
        self.haplotypes = self.haplotypes.to(device)
        return self

    def __repr__(self) -> str:
        return f"Population(pop_size={self.pop_size}, genome={self.genome}, device={self.device})"

class Trait:
    def __init__(self, genome: Genome, population: Population, target_mean: float = 0.0, target_variance: float = 1):
        self.genome: Genome = genome
        self.device: torch.device = population.device
        self.target_mean: float = target_mean
        self.target_variance: float = target_variance

        # Use torch.randn with a generator for reproducibility
        generator = torch.Generator(device=self.device)
        generator.manual_seed(torch.initial_seed())  # Use the seed set by torch.manual_seed()
        raw_effects = torch.randn(genome.n_markers, device=self.device, generator=generator)

        centered_effects = raw_effects - raw_effects.mean()
        dosages = population.haplotypes.sum(dim=1)
        founder_values = torch.einsum('ij,j->i', dosages, centered_effects)
        founder_mean = founder_values.mean()
        founder_var = founder_values.var()

        scaling_factor = torch.sqrt(self.target_variance / founder_var)
        self.effects: torch.Tensor = centered_effects * scaling_factor
        self.intercept: torch.Tensor = (torch.tensor(self.target_mean, device=self.device) - founder_mean).detach()
    def to(self, device: torch.device) -> 'Trait':
        self.device = device
        self.effects = self.effects.to(device)
        self.intercept = self.intercept.to(device)
        return self

    def __repr__(self) -> str:
        return f"Trait(target_mean={self.target_mean}, target_variance={self.target_variance}, device={self.device})"




class SimOps:
    @staticmethod
    def score_population(population: Population, trait: Trait, h2: float = 1.0) -> torch.Tensor:
        dosages = population.haplotypes.sum(dim=1)
        breeding_values = torch.einsum('ij,j->i', dosages, trait.effects)

        bv_var = breeding_values.var()
        if bv_var == 0 or h2 >= 1:
            return breeding_values

        env_variance = (1 - h2) / h2 * bv_var.item()
        env_std = torch.sqrt(torch.tensor(env_variance, device=population.device))
        env_effects = torch.randn_like(breeding_values) * env_std
        return breeding_values + env_effects + trait.intercept


    @staticmethod
    def truncation_selection(population: Population, phenotypes: torch.Tensor, selection_intensity: float, return_indices: bool = False) -> torch.Tensor:
        assert 0 < selection_intensity <= 1, "Selection intensity must be between 0 and 1"
        assert population.haplotypes.shape[0] == phenotypes.shape[0], "Mismatch between population size and phenotypes"

        num_select = max(1, min(int(selection_intensity * phenotypes.shape[0]), phenotypes.shape[0] - 1))
#         print(f'DEBUG: num_select: {num_select}, population size: {phenotypes.shape[0]}')

        _, top_indices = torch.topk(phenotypes, num_select)
        if return_indices:
            return top_indices
        return population.haplotypes[top_indices]



    @staticmethod
    def meiosis(selected_haplotypes: torch.Tensor, num_crossovers: int = 1, num_gametes_per_parent: int = 1) -> torch.Tensor:
        num_parents, ploidy, num_markers = selected_haplotypes.shape

        # Repeat each parent's haplotypes num_gametes_per_parent times
        expanded_haplotypes = selected_haplotypes.repeat_interleave(num_gametes_per_parent, dim=0)

        # The rest of the function remains largely the same, but operates on the expanded haplotypes
        total_gametes = num_parents * num_gametes_per_parent

        crossover_points = torch.randint(1, num_markers, (total_gametes, num_crossovers), device=selected_haplotypes.device, generator=torch.Generator(device=selected_haplotypes.device).manual_seed(torch.initial_seed()))
        crossover_points, _ = torch.sort(crossover_points, dim=1)

        crossover_mask = torch.zeros((total_gametes, num_markers), dtype=torch.bool, device=selected_haplotypes.device)
        crossover_mask.scatter_(1, crossover_points, 1)
        crossover_mask = torch.cumsum(crossover_mask, dim=1) % 2 == 1

        crossover_mask = crossover_mask.unsqueeze(1).expand(-1, ploidy, -1)

        start_chromosome = torch.randint(0, ploidy, (total_gametes, 1), device=selected_haplotypes.device)
        start_mask = start_chromosome.unsqueeze(-1).expand(-1, -1, num_markers)

        final_mask = crossover_mask ^ start_mask.bool()

        offspring_haplotypes = torch.where(final_mask, expanded_haplotypes, expanded_haplotypes.roll(shifts=1, dims=1))

        # Return only the first haplotype for each meiosis event
        return offspring_haplotypes[:, 0, :]
    @staticmethod
    def check_cuda(tensor: torch.Tensor, name: str) -> None:
        print(f"{name} is on: {tensor.device}")


    @staticmethod
    def random_cross(gamete_tensor: torch.Tensor, total_crosses: int) -> torch.Tensor:
        num_gametes, n_markers = gamete_tensor.shape

        # Double the gamete tensor until we have enough for the total crosses
        while num_gametes < 2 * total_crosses:
            gamete_tensor = torch.cat([gamete_tensor, gamete_tensor], dim=0)
            num_gametes *= 2

        # Randomly select gametes for crossing
        gamete_indices = torch.randperm(num_gametes, device=gamete_tensor.device)
        parent1_indices = gamete_indices[:total_crosses]
        parent2_indices = gamete_indices[total_crosses:2*total_crosses]

        # Create the new population haplotype tensor
        new_population = torch.stack([
            gamete_tensor[parent1_indices],
            gamete_tensor[parent2_indices]
        ], dim=1)

        return new_population
    

class SimParams:
    def __init__(self, config):
        self.config = config
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.genome = Genome(config.n_markers)
        # Set the seed
        torch.manual_seed(config.seed)
        np.random.seed(config.seed)
        random.seed(config.seed)
        
        random_parents = torch.rand(config.starting_parents, self.genome.ploidy, self.genome.n_markers, device=self.device)
        random_parents = haplotypes = (random_parents < 0.5).to(torch.int).float()
        self.population = Population(config.pop_size, random_parents, self.genome, self.device)
        
        f1 = self.perform_meiosis(self.population.haplotypes, num_crossovers=3, num_gametes_per_parent=((2000//self.population.haplotypes.shape[0]) *self.population.haplotypes.shape[0] + 100))
        self.create_next_generation(f1, total_crosses=200)
        
        # Burn-in period
        burn_in_trait = Trait(self.genome, self.population)
        for _ in range(100):  # 20 cycles of burn-in
            phenotypes = self.score_population(trait=burn_in_trait)
            selected_indices = self.truncation_selection(selection_intensity=0.9, phenotypes=phenotypes, return_indices=True)
            selected_haplotypes = self.population.haplotypes[selected_indices]
            
            gametes = self.perform_meiosis(selected_haplotypes, num_crossovers=3, num_gametes_per_parent=100)
            self.create_next_generation(gametes, total_crosses=200)
        
        # Create final segregating population
        num_gametes_per_parent = (self.population.haplotypes.shape[0] / config.pop_size) + 2 
        segregating_pop = self.perform_meiosis(self.population.haplotypes, num_crossovers=3, num_gametes_per_parent=100)
        self.create_next_generation(segregating_pop, total_crosses=config.pop_size)
        # Burn-in period with a different seed
        torch.manual_seed(config.seed+1)
        np.random.seed(config.seed+1)
        random.seed(config.seed+1)
        # Create the trait after burn-in
        self.trait = Trait(self.genome, self.population)
        
        # Calculate initial statistics for normalization
        self.initial_phenotypes = self.score_population()
        self.initial_max_phenotype = self.initial_phenotypes.max().item()
        self.initial_phenotype_std = self.initial_phenotypes.std().item()
        print(f"Initial phenotype mean after trait initialization: {self.initial_phenotypes.mean().item()}")
        self.founder_pop = self.population


    def pop_summary(self):
        # Calculate inbreeding coefficient
        allele_freq = self.population.haplotypes.mean(dim=(0, 1))
        expected_heterozygosity = 2 * allele_freq * (1 - allele_freq)
        observed_heterozygosity = (self.population.haplotypes[:, 0, :] != self.population.haplotypes[:, 1, :]).float().mean(dim=0)
        
        # Avoid division by zero and handle NaN values
        valid_loci = expected_heterozygosity != 0
        inbreeding_coeff = torch.zeros_like(expected_heterozygosity)
        inbreeding_coeff[valid_loci] = 1 - (observed_heterozygosity[valid_loci] / expected_heterozygosity[valid_loci])
        
        # Replace NaN values with 0.01
        inbreeding_coeff = torch.where(torch.isnan(inbreeding_coeff), torch.tensor(0.01, device=inbreeding_coeff.device), inbreeding_coeff)
        
        avg_inbreeding_coeff = inbreeding_coeff.mean().item()

        # Calculate MAF
        maf = torch.min(allele_freq, 1 - allele_freq)
        avg_maf = maf.mean().item()

        # Calculate trait mean and genetic variance
        genetic_values = self.score_population(h2=1.0)  # Set h2 to 1.0 to get pure genetic values
        trait_mean = genetic_values.mean().item()
        genetic_variance = genetic_values.var().item()

        return {
            "inbreeding_coefficient": avg_inbreeding_coeff,
            "minor_allele_frequency": avg_maf,
            "trait_mean": trait_mean,
            "genetic_variance": genetic_variance,
            "genetic_values": genetic_values.cpu().numpy()  # Convert to numpy array for easier handling
        }
    
    
    def normalize_reward(self, reward: float) -> float:
        """Normalize the reward based on initial population statistics."""
        normalized_reward = (reward - self.initial_max_phenotype) / self.initial_phenotype_std
        return normalized_reward
    
#     def score_population(self, h2: Optional[float] = None) -> torch.Tensor:
#         h2 = h2 if h2 is not None else self.config.h2
#         return SimOps.score_population(self.population, self.trait, h2)

#     def truncation_selection(self, selection_intensity: Optional[float] = None, phenotypes: Optional[torch.Tensor] = None, return_indices: bool = False) -> torch.Tensor:
#         selection_intensity = selection_intensity if selection_intensity is not None else self.config.selection_intensity
#         if phenotypes is None:
#             phenotypes = self.score_population()
#         return SimOps.truncation_selection(self.population, phenotypes, selection_intensity, return_indices)

    def check_device(self) -> None:
        SimOps.check_cuda(self.population.haplotypes, "Population haplotypes")
        SimOps.check_cuda(self.trait.effects, "Trait effects")
        SimOps.check_cuda(self.trait.intercept, "Trait intercept")

    def to(self, device: torch.device) -> 'SimParams':
        self.device = device
        self.population = self.population.to(device)
        self.trait = self.trait.to(device)
        return self
    def perform_meiosis(self, selected_haplotypes: torch.Tensor, num_crossovers: int = 2, num_gametes_per_parent: int = 1) -> torch.Tensor:
        return SimOps.meiosis(selected_haplotypes, num_crossovers, num_gametes_per_parent)

    def create_next_generation(self, gametes: torch.Tensor, total_crosses: int) -> None:
        new_population_haplotypes = SimOps.random_cross(gametes, total_crosses)
        self.population.haplotypes = new_population_haplotypes
        self.population.pop_size = total_crosses

    def score_population(self, trait=None, h2=None):
        trait = trait if trait is not None else self.trait
        h2 = h2 if h2 is not None else self.config.h2
        return SimOps.score_population(self.population, trait, h2)

    def truncation_selection(self, selection_intensity=None, phenotypes=None, return_indices=False):
        selection_intensity = selection_intensity if selection_intensity is not None else self.config.selection_intensity
        if phenotypes is None:
            phenotypes = self.score_population()
        return SimOps.truncation_selection(self.population, phenotypes, selection_intensity, return_indices)


import torch
import time

def create_sim_instance(device):
    config = SimulationConfig()
    return SimParams(config).to(device)

def run_simulation(sim_instance, num_generations):
    start_time = time.time()
    for _ in range(num_generations):
        phenotypes = sim_instance.score_population()
        selected_indices = sim_instance.truncation_selection(selection_intensity=0.2, phenotypes=phenotypes, return_indices=True)
        selected_haplotypes = sim_instance.population.haplotypes[selected_indices]
        gametes = sim_instance.perform_meiosis(selected_haplotypes, num_crossovers=2, num_gametes_per_parent=50)
        sim_instance.create_next_generation(gametes, total_crosses=sim_instance.config.pop_size)
    end_time = time.time()
    return end_time - start_time

# Create CPU and CUDA instances
cpu_sim = create_sim_instance(torch.device('cpu'))
cuda_sim = create_sim_instance(torch.device('cuda')) if torch.cuda.is_available() else None

# Run simulations
num_generations = 10
cpu_time = run_simulation(cpu_sim, num_generations)
print(f"CPU simulation time: {cpu_time:.2f} seconds")

if cuda_sim is not None:
    cuda_time = run_simulation(cuda_sim, num_generations)
    print(f"CUDA simulation time: {cuda_time:.2f} seconds")
    print(f"Speedup: {cpu_time / cuda_time:.2f}x")
else:
    print("CUDA not available")

Initial phenotype mean after trait initialization: 10.083398818969727
Initial phenotype mean after trait initialization: 10.083398818969727


# Environment

In [ ]:
@dataclass
class SimulationConfig:
    #training
    total_timesteps: int = 2000000
    seed: int = 1
    #breeding sim parameters
    n_markers: int = 1000
    starting_parents: int = 2
    pop_size: int = 1000
    h2: float = 0.3
    max_generations:int = 6
    sparse_reward: bool = True
    budget_per_cycle: int = 200
    starting_budget: int = max_generations * budget_per_cycle
    min_offspring: int = 20
    selection_subpops: int = 2
    fixed_budget: bool = False
    num_crossovers: int = 2

config = SimulationConfig()

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class EnvBasic(gym.Env):
    """A basic breeding simulation environment with configurable action space bins.
    
    Observation Space:
        Population Mean Trait Value (float)
        Population Genetic Variance (float)
        Inbreeding Coefficient (float)
        MAF (float)
        Generations Remaining (float)
        Budget (int)
         
    Action Space:
        Selection Intensity (configurable number of bins between 0.01 and 0.99)
        Number of Crosses to make (10, 20, 30)
    """

    def __init__(self, config, SP):
        super(EnvBasic, self).__init__()

        self.config = config
        self.sim = SP
        self.initial_population = SP.population

        # Create selection intensity bins
        self.selection_intensities = np.linspace(0.01, 0.5, config.si_bins)
        self.budgets = np.linspace(10,30, config.budget_bins)
        print('Action Space Discrete')
        print(f"Selection Intensities: {np.round(self.selection_intensities, 2)}")
        print(f"Budgets: {np.round(self.budgets, 2)}")
        # Define action space
        self.action_space = spaces.MultiDiscrete([config.si_bins, config.budget_bins])
        
        # Define observation space
        self.observation_space = spaces.Box(
            low=np.array([0, 0, 0, 0, 0, 0]),
            high=np.array([np.inf, np.inf, 1, 0.5, config.max_generations, config.starting_budget]),
            dtype=np.float32
        )

        # Environment parameters
        self.max_generations = config.max_generations
        self.starting_budget = config.starting_budget
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # Reset to initial population
        self.sim.population = self.sim.founder_pop
        
        self.current_generation = 0
        self.budget = self.config.starting_budget
        
        observation = self._get_observation()
        info = {}
        return observation, info

    def _get_observation(self):
        summary = self.sim.pop_summary()
        return np.array([
            summary['trait_mean'],
            summary['genetic_variance'],
            summary['inbreeding_coefficient'],
            summary['minor_allele_frequency'],
            self.max_generations - self.current_generation,
            self.budget
        ], dtype=np.float32)
        
    def step(self, action):
        # Get the actions
        selection_intensity = self.selection_intensities[action[0]]
        budget = self.budgets[action[1]]

        # Confirm we can afford the action budget
        if budget > self.budget:
            budget = self.budget  # Reduce budget to what we can afford

        # Phenotype the current population
        phenotypes = self.sim.score_population()

        # Convert selection intensity to number of parents
        num_parents = int(np.ceil(selection_intensity * self.config.pop_size))

        # Select parents based on phenotypes
        selected_indices = self.sim.truncation_selection(selection_intensity=selection_intensity, 
                                                        phenotypes=phenotypes, 
                                                        return_indices=True)
        selected_haplotypes = self.sim.population.haplotypes[selected_indices]

        # Make gametes
        gametes = self.sim.perform_meiosis(selected_haplotypes, 
                                        num_crossovers=self.config.num_crossovers, 
                                        num_gametes_per_parent=100)  # Adjust as needed

        # Make crosses based on budget
        num_crosses = int(budget)
        self.sim.create_next_generation(gametes, total_crosses=num_crosses)

        # Update environment state
        self.current_generation += 1
        self.budget -= budget

        # Get new observation
        observation = self._get_observation()

        # Check if episode is done
        done = (self.current_generation >= self.max_generations) or (self.budget <= 0)

        # Calculate reward (you may want to customize this)
        reward = observation[0]  # Using trait mean as reward for now

        info = {}

        return observation, reward, done, False, info

In [ ]:
cuda_sim = create_sim_instance(torch.device('cuda')) if torch.cuda.is_available() else None
cuda_be =  EnvBasic(config, cuda_sim)


cpu_sim = create_sim_instance('cpu')
cpu_be =  EnvBasic(config, cuda_sim)



In [ ]:
config.si_bins = 3
config.budget_bins = 3
be =  EnvBasic(config, cuda_sim)
be.reset()